In [1]:
import pandas as pd 
import numpy as np

In [2]:
# Pandas >= 0.25 required for this notebook
print(pd.__version__)

1.1.5


## Below is a modified and compressed version of Yuan's work on categories

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
df_cat_hierarchy = pd.read_csv('category_hierarchy.csv', delimiter='|',error_bad_lines=False)
df_items = pd.read_csv('items.csv',sep='|',error_bad_lines=False)
# fill NA values in category column
df_items["categories"] = df_items["categories"].fillna('[4300]')
df_items.head()

,itemID,brand,feature_1,feature_2,feature_3,feature_4,feature_5,categories
0,22665,861,4,0,490,2,66,"[2890, 855, 3908, 3909]"
1,28640,1366,10,1,537,0,101,[4300]
2,13526,1090,10,0,511,0,0,"[3270, 163, 284, 1694, 12, 3837, 2422, 3595, 3..."
3,21399,1090,10,1,511,0,0,[3270]
4,8504,768,4,1,484,0,66,[2470]


In [5]:
# transform categories from str to int list
df_items['categories'] = df_items['categories'].map(lambda x: list(map(int,x.split("[")[1].split("]")[0].split(","))))

In [6]:
# from categories list generate parent_categories
df_indexed = df_cat_hierarchy.set_index(['category'])
df_indexed.head()

,parent_category
category,
0,75
1,1499
2,1082
3,3498
4,1623


In [7]:
flat_hierachy = df_indexed.copy()
flat_hierachy['layer3'] = df_indexed['parent_category'].map(lambda x: df_indexed.loc[x]['parent_category'])
flat_hierachy['layer4'] = flat_hierachy['layer3'].map(lambda x: df_indexed.loc[x]['parent_category'])


flat_hierachy['layer4'].value_counts().head()

3898    3097
2793     109
252       93
2346      81
3624      71
Name: layer4, dtype: int64

In [8]:
# set parent of null as null
df_indexed.loc[4300] = [4300]

## Now flatten the categories
Each row needs to contain only one category, not a list of categories

In [9]:
items_flat = df_items.copy()
items_flat = items_flat.explode('categories')
items_flat.head()

,itemID,brand,feature_1,feature_2,feature_3,feature_4,feature_5,categories
0,22665,861,4,0,490,2,66,2890
0,22665,861,4,0,490,2,66,855
0,22665,861,4,0,490,2,66,3908
0,22665,861,4,0,490,2,66,3909
1,28640,1366,10,1,537,0,101,4300


In [12]:
# parent_categories generate
items_flat['parent_categories'] = items_flat['categories'].map(lambda x : df_indexed.loc[x]['parent_category'].tolist())

In [17]:
items_flat.head()

,itemID,brand,feature_1,feature_2,feature_3,feature_4,feature_5,categories,parent_categories,grandparent_categories,grand_grand_par_categories
0,22665,861,4,0,490,2,66,2890,2832,2838,3898
0,22665,861,4,0,490,2,66,855,1178,2012,3898
0,22665,861,4,0,490,2,66,3908,3898,3898,3898
0,22665,861,4,0,490,2,66,3909,3898,3898,3898
1,28640,1366,10,1,537,0,101,4300,4300,4300,4300


In [18]:
# grandparent_categories generate
items_flat['grandparent_categories'] = items_flat['parent_categories'].map(lambda x : df_indexed.loc[x]['parent_category'].tolist())

In [19]:
# grand_grand_par_categories generate
items_flat['grand_grand_par_categories'] = items_flat['grandparent_categories'].map(lambda x : df_indexed.loc[x]['parent_category'].tolist())

### Now join the category information with the orders

In [21]:
df_orders = pd.read_csv("orders_before_dec.csv", sep = "|")
data = pd.merge(df_orders, items_flat, on="itemID")
print(len(data), "data entries")
data.head()

4890815 data entries


,date,userID,itemID,order,brand,feature_1,feature_2,feature_3,feature_4,feature_5,categories,parent_categories,grandparent_categories,grand_grand_par_categories
0,2020-06-01,38769,3477,1,186,6,0,196,0,45,74,3056,3898,3898
1,2020-06-01,38769,3477,1,186,6,0,196,0,45,4109,3898,3898,3898
2,2020-06-01,38769,3477,1,186,6,0,196,0,45,3867,1543,3898,3898
3,2020-06-01,38769,3477,1,186,6,0,196,0,45,803,1543,3898,3898
4,2020-06-01,38769,3477,1,186,6,0,196,0,45,4053,3898,3898,3898


In [20]:
user_item_group = data.groupby(["userID", "itemID"]).size()
print(user_item_group.describe())
user_item_group.agg(['median'])

count    667301.000000
mean          1.149603
std           0.524784
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max          23.000000
dtype: float64


median    1.0
dtype: float64